# Create predcitors of DeepAR+ using Item Meta data

In [11]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np


In [12]:
# 다른 노트북에서 저장된 변수들을 불러옴
%store -r

In [13]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Hyperparameters

In [14]:
forecastHorizon = 13 # 예측할 시간의 갯수 (480시간 == 20일)
NumberOfBacktestWindows = 2 # Backtest를 하는 횟수
BackTestWindowOffset = 13 # Backtest의 평가(Evaluation)의 갯수
ForecastFrequency = "W"

아래 파라미터의 상세 정보는 [여기](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html) 를 보세요.
![ForecastHorizon](static/imgs/ForecastHorizon.png)
![BackTest](static/imgs/BackTest.png)

#### Predictor algorithms

In [15]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [16]:
print(itme_meta_suffix)

_ItemM


In [17]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + itme_meta_suffix + suffix

In [18]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(PredictorName=prophet_predictorName, 
                                                  AlgorithmArn=prophet_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )




### DeepAR+

In [19]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + itme_meta_suffix + suffix

In [20]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(PredictorName=deeparp_predictorName, 
                                                  AlgorithmArn=deepAR_Plus_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": item_meta_datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )





## Check if the predictors are active


In [21]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [ ]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
item_meta_prophet_predictor_arn = prophet_create_predictor_response["PredictorArn"]
item_meta_deepar_predictor_arn = deeparp_create_predictor_response["PredictorArn"]

In [ ]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_prophet_predictor_arn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= item_meta_deepar_predictor_arn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

## Predictor evaluation

#### Prophet

In [ ]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=prophet_predictor_arn)
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(prophet_metrics)

In [ ]:
# deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
# forecast.describe_predictor(PredictorArn= deepar_predictorArn)

#### DeepAR+

Same as Prophet and ARIMA, now you should look at the metrics from it.

In [ ]:
# DeepAR+ Metrics
deepar_predictor_arn = deepar_predictorArn
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=deepar_predictor_arn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

In [ ]:
%store item_meta_prophet_predictor_arn
%store item_meta_deepar_predictor_arn